In [27]:
import pandas as pd
pd.set_option("display.max_rows", 5)
reviews = pd.read_csv("winemag-data-130k-v2.csv", index_col=0)

print("Setup complete.")

reviews.head()

Setup complete.


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [28]:
# Median of points column
median_points = reviews.points.median()
print(median_points)

88.0


In [29]:
# Countries represented in the dataset
countries = reviews.country.unique()
print(countries)

['Italy' 'Portugal' 'US' 'Spain' 'France' 'Germany' 'Argentina' 'Chile'
 'Australia' 'Austria' 'South Africa' 'New Zealand' 'Israel' 'Hungary'
 'Greece' 'Romania' 'Mexico' 'Canada' nan 'Turkey' 'Czech Republic'
 'Slovenia' 'Luxembourg' 'Croatia' 'Georgia' 'Uruguay' 'England' 'Lebanon'
 'Serbia' 'Brazil' 'Moldova' 'Morocco' 'Peru' 'India' 'Bulgaria' 'Cyprus'
 'Armenia' 'Switzerland' 'Bosnia and Herzegovina' 'Ukraine' 'Slovakia'
 'Macedonia' 'China' 'Egypt']


In [30]:
# How often a country appears in the dataset
reviews_per_country = reviews.country.value_counts()
print(reviews_per_country)

country
US        54504
France    22093
          ...  
China         1
Egypt         1
Name: count, Length: 43, dtype: int64


In [31]:
# Price column with the mean price subtracted
review_price_mean = reviews['price'].mean()
centered_price = reviews.price.map(lambda p: p - review_price_mean)
print(centered_price)

0               NaN
1        -20.363389
            ...    
129969    -3.363389
129970   -14.363389
Name: price, Length: 129971, dtype: float64


In [32]:
# Calculate the points-to-price ratio
reviews['points_to_price_ratio'] = reviews['points'] / reviews['price']

# Find the index of the wine with the highest points-to-price ratio
best_bargain_index = reviews['points_to_price_ratio'].idxmax()

# Get the title of that wine
bargain_wine = reviews.loc[best_bargain_index, 'title']
print(bargain_wine)

Bandit NV Merlot (California)


In [33]:
# Count how many times "tropical" and "fruity" appear in the description column
n_trop = reviews.description.map(lambda desc: "tropical" in desc).sum()
n_fruity = reviews.description.map(lambda desc: "fruity" in desc).sum()

# Create a Series with the results
descriptor_counts = pd.Series([n_trop, n_fruity], index=['tropical', 'fruity'])
print(descriptor_counts)

tropical    3607
fruity      9090
dtype: int64


In [34]:
# Defining start column according to score of 95 or higher counts as 3 stars, a score of at least 85 but less than 95 is 2 stars. Any other score is 1 star. All Canadian wines get 3 stars.

def stars(row):
    if row.country == 'Canada':
       return 3
    elif row.points >= 95:
       return 3
    elif row.points >= 85:
       return 2
    else:
       return 1

reviews["stars"] = reviews.apply(lambda x: stars(x), axis = 1)

star_ratings = reviews["stars"]
print(star_ratings)

0         2
1         2
         ..
129969    2
129970    2
Name: stars, Length: 129971, dtype: int64


In [35]:
# Count how many reviews each person wrote

reviews_written = reviews.groupby('taster_twitter_handle').size()
# or
reviews_written = reviews.groupby('taster_twitter_handle').taster_twitter_handle.count()

print(reviews_written)

taster_twitter_handle
@AnneInVino        3685
@JoeCz             5147
                   ... 
@winewchristina       6
@worldwineguys     1005
Name: taster_twitter_handle, Length: 15, dtype: int64


In [36]:
# Group the data by price and find the maximum points for each price
best_wine_per_price = reviews.groupby('price')['points'].max()

# Sort the Series by price in ascending order
best_wine_per_price_sorted = best_wine_per_price.sort_index()

# This is the desired Series where index is price and values are the max points
best_wine_per_price_sorted


price
4.0       86
5.0       87
          ..
2500.0    96
3300.0    88
Name: points, Length: 390, dtype: int64

In [37]:
# Minimum and maximum prices for each variety of wine? DataFrame whose index is the variety category from the dataset and whose values are the min and max values thereof.
price_extremes = reviews.groupby('variety')['price'].agg(['min', 'max'])
print(price_extremes)

              min   max
variety                
Abouriou     15.0  75.0
Agiorgitiko  10.0  66.0
...           ...   ...
Çalkarası    19.0  19.0
Žilavka      15.0  15.0

[707 rows x 2 columns]


In [39]:
# What are the most expensive wine varieties? Copy of the dataframe from the previous exercise where varieties are sorted in descending order based on minimum price, then on maximum price (to break ties)
sorted_varieties = price_extremes.sort_values(by=['min', 'max'], ascending=[False, False])
print(sorted_varieties)

             min    max
variety                
Ramisco    495.0  495.0
Terrantez  236.0  236.0
...          ...    ...
Vital        NaN    NaN
Zelen        NaN    NaN

[707 rows x 2 columns]


In [40]:
# Create a Series whose index is reviewers and whose values is the average review score given out by that reviewer
reviewer_mean_ratings = reviews.groupby('taster_name')['points'].mean()
print(reviewer_mean_ratings)

taster_name
Alexander Peartree    85.855422
Anna Lee C. Iijima    88.415629
                        ...    
Susan Kostrzewa       86.609217
Virginie Boone        89.213379
Name: points, Length: 19, dtype: float64


In [41]:
# Summary of the range of values.
reviewer_mean_ratings.describe()

count    19.000000
mean     88.233026
           ...    
75%      88.975256
max      90.562551
Name: points, Length: 8, dtype: float64